In [1]:
import openai
import os
import pandas as pd
import random
import time

from dotenv import load_dotenv
from tqdm import tqdm

In [2]:
train = pd.read_csv("data/processed/train.csv")
test = pd.read_csv("data/processed/test.csv")

# Create prompts

for prompting:

Here is an example of a positive movie review, associated with a rating of 10:
...

Here is an example of a negative movie review, associated with a rating of 2:
...

What about the following review? Was its author feeling positive or negative about the movie? Answer with a single word.

In [4]:
prompts = []

# Iterate over the test dataset with a progress bar
for _, test_row in tqdm(test.iterrows(), total=test.shape[0], desc="Generating Prompts"):
    # Select a random positive and negative sentiment review from train
    positive_row = random.choice(train[train['sentiment'] == 'positive'].to_dict('records'))
    negative_row = random.choice(train[train['sentiment'] == 'negative'].to_dict('records'))

    # Randomize the order of positive and negative examples
    examples = random.sample(
        [(f"Here is an example of a positive movie review, associated with a rating of {positive_row['rating']}:\n{positive_row['text']}",
          f"Here is an example of a negative movie review, associated with a rating of {negative_row['rating']}:\n{negative_row['text']}"),
         (f"Here is an example of a negative movie review, associated with a rating of {negative_row['rating']}:\n{negative_row['text']}",
          f"Here is an example of a positive movie review, associated with a rating of {positive_row['rating']}:\n{positive_row['text']}")
        ], 1)[0]

    # Construct the prompt
    prompt = f"""
    {examples[0]}

    {examples[1]}

    What about the following review? Was its author feeling positive or negative about the movie? Answer with a single word.
    {test_row['text']}
    """
    
    prompts.append({
        'review_id': test_row['review_id'],
        'prompt': prompt
    })
    
# Convert the list of prompts into a DataFrame
prompts = pd.DataFrame(prompts)

# Shuffle prompts to randomize the order of examples
prompts = prompts.sample(frac=1).reset_index(drop=True)

Generating Prompts: 100%|██████████| 25000/25000 [14:44<00:00, 28.25it/s]


In [5]:
prompts.to_csv("data/GTP_prompts.csv")

In [6]:
# plot of prompts length

# Send prompts

In [ ]:
load_dotenv("")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
print(os.getenv("OPENAI_API_KEY"))

In [ ]:
print(openai.api_key)

In [ ]:
# Set your OpenAI API key
openai.api_key = 'your_openai_api_key'

# Initialize a list to store responses
responses = []

# Iterate through each prompt and get the model's response, adding progress bar
for index, row in tqdm(prompts_df.iterrows(), total=prompts_df.shape[0], desc="Processing Prompts"):
    prompt = row['prompt']

    try:
        # Send the prompt to the OpenAI API
        response = openai.Completion.create(
            model="text-davinci-003",  # You can use other models like GPT-4 if you have access
            prompt=prompt,
            max_tokens=100,  # Adjust as needed
            temperature=0.7,  # Adjust as needed
            n=5,  # Generate 5 completions
            stop=None
        )
        
        # Collect the responses
        answers = [choice.text.strip() for choice in response.choices]

        # Store the response in the list
        responses.append({
            'review_id': row['review_id'],
            'responses': answers  # A list of 5 different answers
        })

        # Optional: Sleep to prevent rate limiting issues
        time.sleep(1)  # Adjust sleep time as needed based on rate limits

    except Exception as e:
        print(f"Error with prompt {row['review_id']}: {e}")
        responses.append({
            'review_id': row['review_id'],
            'responses': None  # In case of an error, store None as the response
        })

# Convert responses to a DataFrame
results = pd.DataFrame(responses)